In [1]:
import boto3
import pandas as pd
import numpy as np

In [2]:
def download_file_from_s3(bucket, object_name, local_file_name):
    """
    Download a file from S3 to the local file system.

    :param bucket: Name of the S3 bucket
    :param object_name: S3 object name
    :param local_file_name: Local file name to save the downloaded file
    """
    # # When using IAM roles, boto3 retrieves credentials from the instance metadata
    # s3_client = boto3.client('s3')

    #When setting up credentials locally, use the following code
    session = boto3.Session()
    s3_client = session.client('s3')

    s3_client.download_file(bucket, object_name, local_file_name)

bucket_name = 'myukdata'
s3_file_name = 'Original/Member/Member_OppositionPosts.csv'
local_file = 'Member_OppositionPosts.csv'

download_file_from_s3(bucket_name, s3_file_name, local_file)

In [3]:
df = pd.read_csv('Member_OppositionPosts.csv')
df.info()
pd.set_option('display.max_rows', None)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2971 entries, 0 to 2970
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   memberID            2971 non-null   int64  
 1   house               2940 non-null   float64
 2   name                2971 non-null   object 
 3   id                  2971 non-null   int64  
 4   startDate           2971 non-null   object 
 5   endDate             2685 non-null   object 
 6   additionalInfo      2914 non-null   object 
 7   additionalInfoLink  1769 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 185.8+ KB


memberID  house                                               name    id  \
0         2    1.0  Shadow Secretary of State for Culture, Media a...  1186   
1         5    1.0                            Shadow Attorney General    11   
2         7    1.0                  Shadow Minister without Portfolio    17   
3         7    1.0     Shadow Secretary of State for Northern Ireland   873   
4         8    1.0    Shadow Secretary of State for Work and Pensions   866   

             startDate              endDate  \
0  2016-07-05T00:00:00  2016-10-07T00:00:00   
1  1997-06-11T00:00:00  1999-06-15T00:00:00   
2  2005-06-30T00:00:00  2009-05-14T00:00:00   
3  1997-06-11T00:00:00  2001-06-07T00:00:00   
4  2009-01-19T00:00:00  2010-05-06T00:00:00   

                            additionalInfo  \
0  Department for Culture, Media and Sport   
1                Attorney General's Office   
2                   HM Official Opposition   
3                  Northern Ireland Office   
4         Department for Work and Pensions   

                                  additionalInfoLink  
0  https://www.gov.uk/government/organisations/de...  
1  https://www.gov.uk/government/organisations/at...  
2                                                NaN  
3  https://www.gov.uk/government/organisations/no...  
4  https://www.gov.uk/government/organisations/de...

In [4]:
df.drop(columns=['additionalInfo',
                 'additionalInfoLink',
                 'house',
                 'id'], inplace=True)

df.rename(columns={'memberID': 'memberId',
                     'name': 'oppositionName'}, inplace=True)
df['startDate'] = pd.to_datetime(df['startDate'])
df['endDate'] = pd.to_datetime(df['endDate'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2971 entries, 0 to 2970
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   memberId        2971 non-null   int64         
 1   oppositionName  2971 non-null   object        
 2   startDate       2971 non-null   datetime64[ns]
 3   endDate         2685 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 93.0+ KB


In [6]:
#Save the file to S3
import boto3
import io

def upload_df_to_s3(df, bucket, object_name):
    """
    Upload a DataFrame to an S3 bucket as CSV.

    :param df: DataFrame to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name
    :return: True if the DataFrame was uploaded, else False
    """
    # Create a buffer
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)

    # Move to the start of the buffer
    csv_buffer.seek(0)

    # Upload the buffer content to S3
    s3_client = boto3.client('s3')
    try:
        s3_client.put_object(Bucket=bucket, Key=object_name, Body=csv_buffer.getvalue())
    except ClientError as e:
        logging.error(e)
        return False
    return True

bucket_name = 'myukdata'
object_name = 'Preprocessing/Member/MemberOppositionPosts.csv'
upload_df_to_s3(df, bucket_name, object_name)


True